# Lab 4 - recommendations for news services

## Preparation

 * download and unpack the dataset: https://mind201910small.blob.core.windows.net/release/MINDsmall_train.zip
   * read more here: https://learn.microsoft.com/en-us/azure/open-datasets/dataset-microsoft-news
 * [optinal] create a python virtual enviroment
 `python3 -m venv ./recsyslab4`
 * install needed libraries:
 `pip install nltk sklearn`

## Part 1 - data preparation and preprocessing

In [14]:
# import needed packages and libraries

import codecs
from collections import defaultdict
import math
import re
from string import punctuation

import nltk
nltk.download('stopwords')
nltk.download('rslp')

from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

# Chance to improve the performance time with knn algorithms
from sklearn.neighbors import NearestNeighbors

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Piotr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Piotr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping stemmers\rslp.zip.


In [2]:
# defining constants

PATH = './MINDsmall_train'
STOPWORDS = set(stopwords.words('english'))

In [3]:
# loading text metadata

def parse_news_entry(entry):
    news_id, category, subcategory, title, abstract = entry.split('\t')[:5]
    return {
        'news_id': news_id,
        'category': category,
        'subcategory': subcategory,
        'title': title,
        'abstract': abstract
    }

def get_news_metadata():
    with codecs.open(f'{PATH}/news.tsv', 'r', 'UTF-8') as f:
        raw = [x for x in f.read().split('\n') if x]
        parsed_entries = [parse_news_entry(entry) for entry in raw]
        return {x['news_id']: x for x in parsed_entries}

news = get_news_metadata()
news_ids = sorted(list(news.keys()))
news_indices = {x[1]: x[0] for x in enumerate(news_ids)}
print(len(news))

51282


## Part 2 - TF-IDF

In [29]:
# text normalization

def preprocess_text(text):
    # removing punctuation and digits. Converting to lowercase
    text = text.translate(str.maketrans("", "", punctuation+"0123456789")).lower()   
    # tokenization
    text = text.split()
    # removing stopwords
    text = [word for word in text if word not in stopwords.words('english')]
    return text

def stem_texts(corpus):
#     stemmer = RSLPStemmer()
#     stemmer = WordNetLemmatizer.()
    stemmer = LancasterStemmer()
#     stemmer = PorterStemmer()
#     stemmer = SnowballStemmer()
    return [[stemmer.stem(word) for word in preprocess_text(text)] for text in corpus]

texts = [news[news_id]['abstract'] for news_id in news_ids]
stemmed_texts = stem_texts(texts)

In [30]:
# Let's compare some text before and after processing

print(texts[2] + '\n')
print(' '.join(stemmed_texts[2]))

"I think we have a really good team, and a team that can really do some special, good things because that group is very close in there." - Brian Schmetzer

think real good team team real spec good thing group clos bri schmetzer


In [32]:
# create a list of all words in the corpus

def get_all_words_sorted(corpus):
    return sorted([word for word in text for text in corpus])

wordlist = get_all_words_sorted(stemmed_texts)
word_indices = {x[1]: x[0] for x in enumerate(wordlist)}
print(len(wordlist))

NameError: name 'text' is not defined

In [ ]:
# calculate the number of texts in which each of the words appeard
# If it appeared multiple times in one text we still count it only once

def get_document_frequencies(corpus, wordlist):
    # return {word -> count}
    raise NotImplementedError()

document_frequency = get_document_frequencies(stemmed_texts, wordlist)

In [ ]:
# obliczamy liczbe wystapien kazdego slowa w kazdym tekscie
# calculate the number of word occurrences in each text

def get_term_frequencies(corpus, news_indices):
    # return {news_id -> {word -> count}}
    raise NotImplementedError()

term_frequency = get_term_frequencies(stemmed_texts, news_indices)

In [ ]:
# check the results

term_frequency[news_ids[2]]

In [ ]:
# calculate the tf_idf metric

def calculate_tf_idf(term_frequency, document_frequency, corpus_size):
    # return {news_id -> {word -> tf_idf}}
    raise NotImplementedError()

tf_idf = calculate_tf_idf(term_frequency, document_frequency, len(news_ids))

In [ ]:
# check the results

tf_idf[news_ids[2]]

## Part 3 - text similarity

In [ ]:
# calculate the distance between 2 elements/texts
# test different metrics and chose one

def calculate_distance(tf_idf, id1, id2):
    raise NotImplementedError()

calculate_distance(tf_idf, news_ids[2], news_ids[1])

In [ ]:
# for a given text find k most similar ones
# remember about the proper sorting order for used metric
# remember to not include the currently checked text. 
# It would always be the most similar because it's identical

def get_k_most_similar_news(tf_idf, n_id, k):
    raise NotImplementedError()

def print_k_most_similar_news(tf_idf, n_id, k, corpus, news_indices):
    similar = get_k_most_similar_news(tf_idf, n_id, k)
    print(f'id: {n_id}, text: {corpus[news_indices[n_id]]}')
    print(f'\n{k} most similar:')
    for s_id in similar:
        print(f'\nid: {s_id}, text: {corpus[news_indices[s_id]]}')

print_k_most_similar_news(tf_idf, news_ids[42337], 5, texts, news_indices)